In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
np.set_printoptions(suppress=True, precision=3)

from deepc_utils import *

from select_deepc.deepc_controller import LkSelector, CosineDistances, IsoMapEmbeddedDistances, RandomSelector
from select_deepc.hankel_generation import HankelMatrixGenerator

import warnings
from itertools import product
import matplotlib.pyplot as plt
import pickle

from tqdm import tqdm

plt.rcParams.update(
        {
            "text.usetex": True,
            "font.family": "serif",
            "font.sans-serif": "Times",
            "font.size": 10,
        }
    )


PAPER_COL_WIDTH_IN = (6.75 - 0.25) / 2
PAPER_CONTENT_WIDTH_IN = 6.75
PAPER_COL_WIDTH_CM = PAPER_COL_WIDTH_IN * 2.54
PAPER_CONTENT_WIDTH_CM = PAPER_CONTENT_WIDTH_IN * 2.54


color_iso = "salmon"
color_l1 = "chocolate"
color_rand = "orchid"

In [ ]:
with open("data/prediction_validation/residuals_affine_1past_fixed_rnd.pkl", "rb") as file:
    (
        rand_residuals_affine,
        l1_residuals_affine,
        # cos_residuals_affine,
        iso_4_residuals_affine,
        # iso_7_residuals_affine
    ) = pickle.load(file)

with open("data/prediction_validation/residuals_1past_fixed_rnd.pkl", "rb") as file:
    (
        rand_residuals,
        l1_residuals,
        # cos_residuals,
        iso_4_residuals,
        # iso_7_residuals
    ) = pickle.load(file)

In [ ]:
pid_trajectory = load_data_from_folder("data/rocket/dataset_pid", "pid")
iid_trajectories = load_data_from_folder("data/rocket/dataset_iid", "iid")
random_walk_trajectories = load_data_from_folder("data/rocket/dataset_random_walk", "random_walk")
random_walk_trajectories_2 = load_data_from_folder("data/rocket/dataset_random_walk_2", "random_walk_2")
# prbs_trajectory = TrajectoryDataSet(
#     [TrajectoryData(
#         np.loadtxt("../DeePC-HUNT/examples/data/rocket_yd.csv", delimiter=","),
#         np.loadtxt("../DeePC-HUNT/examples/data/rocket_ud.csv", delimiter=","),
#     )],
#     "prbs",
# )
cl_traj = TrajectoryDataSet(
    [TrajectoryData(
        np.loadtxt("data/rocket/cl_traj_x.csv", delimiter=","),
        np.loadtxt("data/rocket/cl_traj_u.csv", delimiter=","),
    )], "cl_traj"
)

In [ ]:
hankel_generator = HankelMatrixGenerator(2, 30)
validation_set = hankel_generator.generate_hankel_matrices(pid_trajectory)

random_walk_hankel = hankel_generator.generate_hankel_matrices(random_walk_trajectories)
random_walk_2_hankel = hankel_generator.generate_hankel_matrices(random_walk_trajectories_2)
iid_hankel = hankel_generator.generate_hankel_matrices(iid_trajectories)

num_validation_trajs = validation_set[0].shape[-1]

In [ ]:
l1_sel = LkSelector()
cos_sel = CosineDistances()
rand_sel = RandomSelector()

In [ ]:
def predict(u_val, y_val, H_u_sel, Y_p, Y_f, dims, use_affine=False, return_jac=False):
        H = (
            np.vstack((H_u_sel, Y_p, np.ones((1, Y_p.shape[-1]))))
            if use_affine
            else np.vstack((H_u_sel, Y_p))
        )
        z = (
            np.append(np.append(u_val, y_val[:dims.T_past*dims.p]), [1])
            if use_affine 
            else np.append(u_val, y_val[:dims.T_past*dims.p])
        )
        jac = np.linalg.pinv(H) 
        pred = Y_f @ jac @ z
        return (pred, Y_f, jac) if return_jac else pred

def test_selector(selector, H_u, H_y, dims: DeePCDims, num_idcs, use_affine=False):
    res = []
    for u_val, y_val in zip(validation_set[0].T, validation_set[1].T):
        idcs, _ = selector(u_val.reshape(-1,1), y_val.reshape(-1,1), H_u, H_y, None)
        idcs = idcs[:num_idcs]
        H_u_sel, H_y_sel = H_u[:, idcs], H_y[:, idcs]
        Y_p, Y_f = H_y_sel[:dims.T_past*dims.p, :], H_y_sel[dims.T_past*dims.p:, :]

        pred = predict(u_val, y_val, H_u_sel, Y_p, Y_f, dims, use_affine=use_affine)

        curr_res = np.linalg.norm(y_val[dims.T_past*dims.p:].reshape(-1) - pred.reshape(-1))**2
        res.append(curr_res)
    return res

In [ ]:
dims = DeePCDims(2, 30, 6, 3)
iso_4_iid = IsoMapEmbeddedDistances(iid_trajectories, dims, n_neighbors=4)
iso_4_rw = IsoMapEmbeddedDistances(random_walk_trajectories, dims, n_neighbors=4)
# iso_4_rw2 = IsoMapEmbeddedDistances(random_walk_trajectories_2, dims, n_neighbors=4)
# iso_4_rw_pid = IsoMapEmbeddedDistances(random_walk_trajectories + pid_trajectory, dims, n_neighbors=4)
# iso_4_rw2_pid = IsoMapEmbeddedDistances(random_walk_trajectories_2 + pid_trajectory, dims, n_neighbors=4)

# dataset_map = {
#     "iid": (iid_trajectories, iso_4_iid, iid_hankel),
#     "rw": (random_walk_trajectories, iso_4_rw, random_walk_hankel),
#     "rw2": (random_walk_trajectories_2, iso_4_rw2, random_walk_2_hankel),
# }

# Jacobians

In [ ]:
for hankel in [random_walk_hankel, random_walk_2_hankel, iid_hankel]:
    h = np.vstack(hankel)
    print(h.shape)
    mean = np.mean(h, axis=1)
    cov = np.cov(h) + 9e-1*np.eye(h.shape[0])
    print(cov.shape)
    # print(cov)
    print(np.log(np.linalg.det(cov)))

In [ ]:
def test_jacobian(dataset, selector, hankels, affine=False):
    test_traj_u, test_traj_y = validation_set[0][:, 512], validation_set[1][:, 512]
    H_u, H_y = hankels
    idcs, _ = selector(test_traj_u.reshape(-1), test_traj_y.reshape(-1), hankels[0], hankels[1], None)
    idcs = idcs[:250]
    H_u_sel, H_y_sel = H_u[:, idcs], H_y[:, idcs]
    Y_p, Y_f = H_y_sel[:dims.T_past*dims.p, :], H_y_sel[dims.T_past*dims.p:, :]

    pred, Y_f, jac = predict(test_traj_u, test_traj_y, H_u_sel, Y_p, Y_f, dims, affine, True)
    im = plt.imshow(np.log(np.abs(Y_f@jac)), interpolation="nearest", aspect="auto")
    # im = plt.imshow(Y_f@jac, interpolation="nearest", aspect="auto")
    plt.colorbar(im)
    plt.title(f"{dataset.dataset_name} jacobian")
    plt.savefig(f"figures/jac_{dataset.dataset_name}{'_affine' if affine else ''}.pdf")
    plt.show()
    return jac

jac_iid = test_jacobian(*dataset_map["iid"], False)
jac_rw = test_jacobian(*dataset_map["rw"], False)
jac_rw2 = test_jacobian(*dataset_map["rw2"], False)
jac_iid_aff = test_jacobian(*dataset_map["iid"], True)
jac_rw_aff = test_jacobian(*dataset_map["rw"], True)
jac_rw2_aff = test_jacobian(*dataset_map["rw2"], True)

In [ ]:
im = plt.imshow(np.log(np.abs(jac_rw_aff[:, :-1] - jac_rw)), interpolation="nearest", aspect="auto")
plt.colorbar(im)
plt.savefig("figures/iid_diff.pdf")

In [ ]:
affine_stack = np.vstack(
    (
        jac_rw_aff[:,-1].reshape(1,-1),
        jac_iid_aff[:,-1].reshape(1,-1),
        jac_rw2_aff[:,-1].reshape(1,-1),
    )
)
print(affine_stack.shape)
im = plt.imshow(affine_stack, interpolation="nearest", aspect="auto")
plt.colorbar(im)
plt.savefig("figures/jac_affine_comparison.pdf")


# Prediction Validation

In [ ]:
# rand_residuals_affine = []
# l1_residuals_affine = []
# iso_4_residuals_affine = []
# cos_residuals_affine = []
# iso_7_residuals_affine = []
# iso_11_residuals_affine = []
rand_res_add_m_s = []
l1_res_add_m_s = []
iso_4_res_add_m_s = []
# rand_residuals = []
# l1_residuals = []
# cos_residuals = []
# iso_4_residuals = []
# iso_7_residuals = []

dims = DeePCDims(2, 30, 6, 3)

# for dataset, iso_embedder in zip(
#     [random_walk_trajectories, random_walk_trajectories_2, iid_trajectories],
#     [iso_4_rw, iso_4_rw2, iso_4_iid],
#     # [None, None, None],
# ):
for dataset, iso_embedder in zip(
    [random_walk_trajectories, iid_trajectories],
    [iso_4_rw, iso_4_iid],
    # [None, None, None],
):
# for dataset, iso_embedder in zip(
#     [random_walk_trajectories + pid_trajectory, random_walk_trajectories_2 + pid_trajectory],
#     [iso_4_rw_pid, iso_4_rw2_pid],
#     # [None, None, None],
# ):
    print(f"testing on {dataset.dataset_name}")

    hankel = hankel_generator.generate_hankel_matrices(dataset)

    # with warnings.catch_warnings():
    #     warnings.simplefilter("ignore")
    #     iso_embedder_7 = IsoMapEmbeddedDistances(dataset, dims, n_neighbors=7)
    #     iso_embedder_11 = IsoMapEmbeddedDistances(dataset, dims, n_neighbors=11)

    test_idcs = np.logspace(2, np.log10(hankel[0].shape[-1]), num=7, base=10, dtype=int)
    print(f"testing for sizes {test_idcs}")
    for num_idcs in tqdm(test_idcs):
        for selector, residual_list, use_affine in [
            # (rand_sel, rand_residuals, False),
            (rand_sel, rand_res_add_m_s, True),
            # (l1_sel, l1_residuals, False),
            (l1_sel, l1_res_add_m_s, True),
            # (cos_sel, cos_residuals, False),
            # (cos_sel, cos_residuals_affine, True),
            # (iso_embedder, iso_4_residuals, False),
            (iso_embedder, iso_4_res_add_m_s, True),
            # (iso_embedder_7, iso_7_residuals, False),
            # (iso_embedder_7, iso_7_residuals_affine, True),
            # (iso_embedder_11, iso_11_residuals_affine, True),
        ]:
            residual = test_selector(selector, hankel[0], hankel[1], dims, num_idcs, use_affine)
            residual = np.array(residual)
            residual_list.append((residual, dataset.dataset_name, num_idcs))

In [ ]:
for i, xmax in zip(
    range(14),
    [10000, 1000, 500, 200, 100, 100, 100, 1000, 100, 50, 20, 10, 10, 10],
):
    plt.hist(rand_res_add_m_s[i][0], bins=100)
    plt.hist(l1_res_add_m_s[i][0], bins=100, alpha=0.5)
    plt.hist(iso_4_res_add_m_s[i][0], bins=100, alpha=0.5)
    # plt.ylim(None,100)
    # plt.xlim(0,xmax)
    plt.show()

In [ ]:
rand_res_add_m_s = rand_residuals_affine[22:]
l1_res_add_m_s = l1_residuals_affine[22:]
iso_res_add_m_s = iso_4_residuals_affine[21:]

In [ ]:
iso_res_add_m_s

In [ ]:
with open("data/prediction_validation/residuals_mean_cov.pkl", "rb") as file:
    (
        rand_res_add_m_s,
        l1_res_add_m_s,
        iso_res_add_m_s,
    ) = pickle.load(file)

In [ ]:
with open(
    "data/prediction_validation/residuals_mean_cov.pkl", "wb"
) as file:
    pickle.dump(
        (
            rand_res_add_m_s,
            l1_res_add_m_s,
            iso_res_add_m_s,
        ), 
        file,
    )

# Plotting the Validation

In [ ]:
dataset_names = ["random_walk", "iid"]
display_names = ["Random Walk Data", "IID Data"]

fig, axs = plt.subplots(1, 2, sharey=True, figsize=(PAPER_COL_WIDTH_IN, 1.75), tight_layout=True)

for name, ax, disp_name in zip(dataset_names, axs, display_names):
    for selname, residual_vec, color in zip(
        ["Random", r"$L_1$", "Isomap"],
        [
            rand_residuals_affine,
            l1_residuals_affine,
            iso_4_residuals_affine,
        ],
        [color_rand, color_l1, color_iso],
        # plt.rcParams["axes.prop_cycle"].by_key()["color"],
    ):
        res = [
            (residual, num_idcs)
            for residual, dname, num_idcs in residual_vec
            if dname == name
        ]

        arr = np.array(res)
        if name == "iid":
            ax.loglog(
                arr[:, 1],
                arr[:, 0] / num_validation_trajs,
                "-",
                c=color,
                label=f"{selname}",
            )
        else:
            ax.loglog(arr[:, 1], arr[:, 0] / num_validation_trajs, "-", c=color)

    ax.set_title(disp_name, fontsize="small")
    # ax.set_xlim(200, None)

# axs[0].set_ylim(None, 1e3)
axs[1].set_ylim(3e0, 1e2)
# axs[2].set_ylim(None, 1e3)
axs[0].set_ylabel(r"$\|y_\textrm{f} - \hat{y}_\textrm{f} \|_2$")
fig.text(
    0.55, 0.04, r"$N_\textrm{cols}$", va="center", ha="center", fontsize=plt.rcParams["axes.labelsize"]
)
fig.suptitle("Predictive Accuracy on Rocket Data Sets", y=0.85)

# fig.supxlabel(r"$N_\textrm{cols}$")
# axs[1].set_xlabel(r"$N_\textrm{cols}$")
fig.legend(fontsize="small", bbox_to_anchor=(0.96,0), ncols=3, title=r"\textbf{Selection Method}")
fig.tight_layout()
fig.savefig(f"figures/prediction_validation/affine_only.pdf", bbox_inches="tight")
plt.show()

In [ ]:

dataset_names = ["random_walk", "iid"]
display_names = ["Random Walk Data", "IID Data"]

fig, axs = plt.subplots(1, 2, sharey=True, figsize=(PAPER_COL_WIDTH_IN, 1.5), tight_layout=True)

for name, ax, disp_name in zip(dataset_names, axs, display_names):
    for selname, residual_vec, color in zip(
        ["Random", r"$L_1$", "Isomap"],
        [
            rand_res_add_m_s,
            l1_res_add_m_s,
            iso_res_add_m_s,
        ],
        [color_rand, color_l1, color_iso],
        # plt.rcParams["axes.prop_cycle"].by_key()["color"],
    ):
        res = [
            (mean, std, num_idcs)
            for (mean, std), dname, num_idcs in residual_vec
            if dname == name
        ]

        arr = np.array(res)
        if name == "iid":
            ax.loglog(
                arr[:, 2],
                arr[:, 0],
                "-",
                c=color,
                label=f"{selname}",
            )
        else:
            ax.loglog(arr[:, 2], arr[:, 0], "-", c=color)

        ax.fill_between(arr[:,2], arr[:,0]- arr[:,1], arr[:,0] + arr[:,1], color=color, alpha=0.1)

    ax.set_title(disp_name)
    # ax.set_xlim(200, None)

axs[0].set_ylim(None, 1e2)
# axs[1].set_ylim(3e0, 1e2)
# axs[2].set_ylim(None, 1e3)
axs[0].set_ylabel("Cum. Res.")
fig.text(
    0.55, 0.04, r"$N_\textrm{cols}$", va="center", ha="center", fontsize=plt.rcParams["axes.labelsize"]
)

# fig.supxlabel(r"$N_\textrm{cols}$")
# axs[1].set_xlabel(r"$N_\textrm{cols}$")
fig.legend(fontsize="small", bbox_to_anchor=(0.96,0), ncols=3, title=r"\textbf{Selection Method}")
fig.tight_layout()
fig.savefig(f"figures/prediction_validation/affine_only.pdf", bbox_inches="tight")
plt.show()

In [ ]:
from matplotlib.lines import Line2D
from matplotlib.patches import FancyBboxPatch

dataset_names = ["random_walk", "iid"]

fig, ax = plt.subplots(1, 1, sharey=True, figsize=(PAPER_COL_WIDTH_IN, 2), tight_layout=True)

for name in dataset_names:
    for selname, residual_vec, color in zip(
        [
            "Random",
            r"$L_1$",
            "Isomap",
        ],
        [
            rand_residuals_affine,
            l1_residuals_affine,
            iso_4_residuals_affine,
        ],
        [
            color_rand,
            color_l1,
            color_iso,
        ]
        # plt.rcParams["axes.prop_cycle"].by_key()["color"],
    ):
        res = [
            (residual, num_idcs)
            for residual, dname, num_idcs in residual_vec
            if dname == name
        ]

        arr = np.array(res)
        if name == "iid":
            ax.loglog(
                arr[:, 1],
                arr[:, 0] / num_validation_trajs,
                "--",
                c=color,
                label=f"{selname}",
            )
        else:
            ax.loglog(arr[:, 1], arr[:, 0] / num_validation_trajs, "-", c=color)

    # ax.set_title(name.replace("_", " ").title())
    # ax.set_xlim(200, None)

# axs[0].set_ylim(None, 1e3)
ax.set_ylim(3e0, 1e2)
# axs[2].set_ylim(None, 1e3)
ax.set_ylabel("Cumulative Resdiual")
fig.text(
    0.55, 0.04, r"$N_\textrm{cols}$", va="center", ha="center", fontsize=plt.rcParams["axes.labelsize"]
)

# fig.supxlabel(r"$N_\textrm{cols}$")
# axs[1].set_xlabel(r"$N_\textrm{cols}$")
fig.legend(
    [
        Line2D([0], [0], color=color_l1, linestyle="", marker=".", markersize=15),
        Line2D([0], [0], color=color_iso, linestyle="", marker=".", markersize=15),
        Line2D([0], [0], color=color_rand, linestyle="", marker=".", markersize=15),
    ],
    [
        r"$L_1$",
        "Isomap",
        "Random Selection"
    ],
    fontsize="small",
    ncols=3,
    bbox_to_anchor=(1, 0),
    title=r"\textbf{Selection Method}",
    frameon=False,
)
legend = fig.legend(
    [
        Line2D([0], [0], color="black", linestyle="--", marker=""),
        Line2D([0], [0], color="black", linestyle="-", marker=""),
    ],
    [
        "IID Data",
        "Random Walk Data",
    ],
    fontsize="small",
    ncols=2,
    frameon=False,
    bbox_to_anchor=(.92,-.2),
)
print(legend.get_frame().get_boxstyle())

rect = FancyBboxPatch(
    (20, 7),  # Starting coordinates (x, y)
    210,  # Width of the box
    43,  # Height of the box
    linewidth=1.5,  # Line width
    edgecolor="lightgrey",  # Box edge color
    facecolor="none",  # Box background color (transparent)
    boxstyle="round, rounding_size=3",
    zorder=10,  # Ensure the box is on top of the plot elements
)

# Add the rectangle patch to the figure, outside the plot area
fig.patches.append(rect)

# fig.legend(fontsize="small", bbox_to_anchor=(0.8,0), ncols=3, title=r"\textbf{Selection Method}")
# fig.tight_layout()
fig.savefig(f"figures/prediction_validation/affine_only_single_plot.pdf", bbox_inches="tight")
plt.show()

In [ ]:
dataset_names = ["random_walk", "random_walk_2", "iid"]

fig, axs = plt.subplots(1, 3, sharey=False, figsize=(6, 3))

for name, ax in zip(dataset_names, axs):
    for selname, residual_vec, color in zip(
        ["Random", r"$l_1$", "Isomap", "iso 7", "iso 11"],
        [
            rand_residuals_affine,
            l1_residuals_affine,
            iso_4_residuals_affine,
            iso_7_residuals_affine,
            iso_11_residuals_affine
        ],
        plt.rcParams["axes.prop_cycle"].by_key()["color"],
    ):
        res = [
            (residual, num_idcs)
            for residual, dname, num_idcs in residual_vec
            if dname == name
        ]

        arr = np.array(res)
        if name == "iid":
            ax.loglog(arr[:, 1], arr[:, 0]/num_validation_trajs, "-", c=color, label=f"{selname}")
        else:
            ax.loglog(arr[:, 1], arr[:, 0]/num_validation_trajs, "-", c=color)

    ax.set_title(name.replace("_", " ").title())
    # ax.set_xlim(200, None)

axs[0].set_ylim(None,1e3)
axs[1].set_ylim(None,1e2)
axs[2].set_ylim(None,1e3)
axs[0].set_ylabel("Cumulative Resdiual")
axs[1].set_xlabel(r"$N_\textrm{cols}$")
axs[2].legend()
fig.tight_layout()
fig.savefig(f"figures/prediction_validation/affine_only_all_selectors.pdf")
plt.show()

In [ ]:
dataset_names = ["random_walk", "random_walk_2", "iid"]

fig, axs = plt.subplots(1,3, sharey=True, figsize=(5.6,3))

for name, ax in zip(dataset_names, axs):
    for selname, residual_vec, color in zip(
        ["Random", r"$l_1$", "Isomap"],
        [rand_residuals, l1_residuals, iso_4_residuals],
        plt.rcParams["axes.prop_cycle"].by_key()["color"],
    ):
        if selname in ["cos", "iso 7"]:
            continue

        res = [(residual, num_idcs) for residual, dname, num_idcs in residual_vec if dname == name]

        arr = np.array(res)
        if name == "iid":
            ax.loglog(arr[:, 1], arr[:,0], label=selname, c=color)
        else:
            ax.loglog(arr[:, 1], arr[:,0], c=color)

    for selname, residual_vec, color in zip(
        ["Random", r"$l_1$", "Isomap"],
        [rand_residuals_affine, l1_residuals_affine, iso_4_residuals_affine],
        plt.rcParams["axes.prop_cycle"].by_key()["color"],
    ):
        if selname in ["cos", "iso 7"]:
            continue

        res = [(residual, num_idcs) for residual, dname, num_idcs in residual_vec if dname == name]

        arr = np.array(res)
        if name == "iid":
            ax.loglog(arr[:, 1], arr[:,0], '--', c=color, label=f"{selname} affine")
        else:
            ax.loglog(arr[:, 1], arr[:,0], '--', c=color)

    ax.set_title(name.replace("_", " ").title())
    # ax.set_xlim(90,1200)

axs[0].set_ylim(None, 1e8)
axs[0].set_ylabel("Cumulative Resdiual")
axs[1].set_xlabel(r"$N_\textrm{cols}$")
axs[2].legend()
fig.tight_layout()
fig.savefig(f"figures/prediction_validation/combined.pdf")
plt.show()

In [ ]:
with open("data/prediction_validation/residuals_affine_newest_2past.pkl", "wb") as file:
    pickle.dump(
        [
            rand_residuals_affine,
            l1_residuals_affine,
            # cos_residuals_affine,
            iso_4_residuals_affine,
            # iso_7_residuals_affine
         ],
        file
    )

# with open("data/prediction_validation/residuals_newest_2past.pkl", "wb") as file:
#     pickle.dump(
#         [
#             rand_residuals,
#             l1_residuals,
#             # cos_residuals,
#             iso_4_residuals,
#             # iso_7_residuals,
#         ],
#         file,
#     )

In [ ]:
rand_residuals_affine

In [ ]:
relative_contrast_fixed = []
for t_fut in tqdm(np.logspace(0, 2, num=7, base=10, dtype=int)):
    test_dims = DeePCDims(2, t_fut, 6, 3)
    curr_val_set = HankelMatrixGenerator(
        test_dims.T_past,
        test_dims.T_fut
    ).generate_hankel_matrices(pid_trajectory)
    curr_hankels = HankelMatrixGenerator(
        test_dims.T_past, test_dims.T_fut
    ).generate_hankel_matrices(random_walk_trajectories_2)

    test_traj_u, test_traj_y = curr_val_set[0][:, 700], curr_val_set[1][:, 700]
    curr_test_traj_u = test_traj_u[:(test_dims.T_past+test_dims.T_fut)*test_dims.m].reshape(-1,1)
    curr_test_traj_y = test_traj_y[:(test_dims.T_past+test_dims.T_fut)*test_dims.p].reshape(-1,1)
    test_selector = IsoMapEmbeddedDistances(
        random_walk_trajectories_2,
        test_dims,
        #n_components=6+test_dims.T_fut*test_dims.m,
    )
    # test_selector = LkSelector()
    idcs, norms = test_selector(
        curr_test_traj_u,
        curr_test_traj_y,
        curr_hankels[0],
        curr_hankels[1],
        None,
    )
    min_norm = np.min(norms)
    max_norm = np.max(norms)
    print(max_norm)
    print(min_norm)
    relative_contrast_fixed.append((max_norm - min_norm)/min_norm)

In [ ]:
np.savetxt("data/relative_contrast_iso/iso_varying.csv", relative_contrast, delimiter=",")
np.savetxt("data/relative_contrast_iso/iso_fixed.csv", relative_contrast_fixed, delimiter=",")
np.savetxt("data/relative_contrast_iso/l1.csv", relative_contrast_l1, delimiter=",")

In [ ]:
relative_contrast_fixed = np.loadtxt("data/relative_contrast_iso/iso_fixed.csv")
relative_contrast = np.loadtxt("data/relative_contrast_iso/iso_varying.csv")
relative_contrast_l1 = np.loadtxt("data/relative_contrast_iso/l1.csv")

fig, axs = plt.subplots(1,1, figsize=(PAPER_COL_WIDTH_IN, 2), sharex=True)
axs.scatter(9*np.logspace(0, 2, num=7, base=10, dtype=int)[:6], relative_contrast_fixed, color=color_iso)
axs.scatter(9*np.logspace(0, 2, num=7, base=10, dtype=int)[:6], relative_contrast, color="darkred", zorder=0)
axs.scatter(9*np.logspace(0, 2, num=7, base=10, dtype=int)[:6], relative_contrast_l1, color=color_l1)
# axs.legend(["Isomap ($n_d$ fixed)", "Isomap ($n_d = 75$)", r"$L_1$"])
axs.set_ylabel(r"Relative Contrast $\Delta$")
axst = axs.twinx()
axst.ticklabel_format(
    axis="y",
    style="sci",
    scilimits=(2,3),
)
axst.plot(np.linspace(0,432), 6 + np.linspace(0,432)*3, "--", alpha=0.75, zorder=0, color="darkred")
axst.plot(np.linspace(0,432), 75*np.ones(50), "--", alpha=0.75, zorder=0, color="salmon")
axs.set_xlabel(r"Trajectory Dimensionality")
axst.set_ylabel("Emb. Dim. (dashed)")
fig.tight_layout()
fig.legend(["Isomap ($n_d = 75$)", "Isomap ($n_d$ varying)", r"$L_1$"], bbox_to_anchor=(.822,.09), ncols=2, fontsize="small")
fig.savefig("figures/relative_contrast_iso.pdf", bbox_inches="tight")